# 거래소에서 option data 가져와보기

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import time
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo  # Python 3.9 이상

from typing import List, Tuple 
from tqdm import tqdm

import requests

In [2]:
from pathlib import Path

CWD_PATH = Path.cwd()
DATA_PATH = CWD_PATH / 'data'
BACKUP_PATH = CWD_PATH / 'backup'
OUTPUT_PATH = CWD_PATH / 'output'

In [3]:
CWD_PATH

WindowsPath('c:/Users/chlje/VSCodeProjects/KAIST_MFE/BAF645파생거래전략/final_hw')

In [4]:
## custom libs

from krx_config import API_URL, HEADERS, PAYLOAD_TEMPLATE, PAYLOAD_OPTION_HOME, HEADERS_OPTION_HOME, PAYLOAD_TEMPLATE2

## API scraping code

### Prompt 1

I need to write code to scrape data from KRX(한국거래소) website. The website uses POST to get daily snapshot of all the options traded on that day. 

Below is the information for its endpint (getJsonData.cmd) and what are in the headers and what the payload is and what the response looks like. 

Write Python code that takes start date and end date as an input and iteratively get the option price data from the api. 

You should make the requests to look like human by setting web browser user-agent and other stuff that can be detected by site admin. 

Although, you don't have to use dynamic scraping like selenium because as you can see, we're communicating with an API in restful way, with POST method. 

When you write the code, make the code modular and keep the function document as simple as possible. Never make verbose docstring that can bloat the code with unnecessary details. 

Below are the information that you need to write the correct API call scraper. 

In [5]:
def generate_date_range(start_date: str, end_date: str) -> List[str]: 
    """
    종료일(end_date)부터 시작일(start_date)까지의 비주말(평일) 날짜를 YYYYMMDD 형식으로 반환합니다.
    날짜는 한국 서울 시간대 기준입니다.

    Args:
        start_date (str): 시작 날짜 ('YYYY-MM-DD' 형식).
        end_date (str): 종료 날짜 ('YYYY-MM-DD' 형식).

    Returns:
        List[str]: 내림차순으로 정렬된 비주말 날짜 리스트 ('YYYYMMDD' 형식).
    """
    tz = ZoneInfo('Asia/Seoul')
    start = datetime.strptime(start_date, "%Y-%m-%d").replace(tzinfo=tz)
    end = datetime.strptime(end_date, "%Y-%m-%d").replace(tzinfo=tz)
    delta = end - start
    date_list = []
    for i in range(delta.days + 1):
        current_date = end - timedelta(days=i)
        if current_date.weekday() < 5:  # 0-4는 월요일~금요일
            date_list.append(current_date.strftime("%Y%m%d"))
    return date_list

def fetch_option_data(session: requests.Session, trade_date: str) -> pd.DataFrame:
    """
    Fetches option data for a specific trade date.

    Args:
        session (requests.Session): The requests session with headers set.
        trade_date (str): Trade date in 'YYYYMMDD' format.

    Returns:
        pd.DataFrame: DataFrame containing option data for the trade date.
    """
    payload = PAYLOAD_TEMPLATE.copy()
    payload["trdDd"] = trade_date

    response = session.post(API_URL, data=payload)
    response.raise_for_status()

    data = response.json()

    if "output" not in data:
        return pd.DataFrame()

    return pd.DataFrame(data["output"])

def scrape_krx_option_data(start_date: str, end_date: str) -> pd.DataFrame:
    """
    Scrapes KRX option data between start_date and end_date.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: Combined DataFrame containing option data for all dates.
    """
    dates = generate_date_range(start_date, end_date)
    all_data = []

    with requests.Session() as session:
        session.headers.update(HEADERS)
        for date in tqdm(dates, desc="Fetching data"):
            try:
                daily_data = fetch_option_data(session, date)
                if not daily_data.empty:
                    daily_data['Trade_Date'] = datetime.strptime(date, "%Y%m%d").date()
                    all_data.append(daily_data)
                time.sleep(1)  # Delay to mimic human behavior
            except requests.HTTPError as http_err:
                print(f"HTTP error for date {date}: {http_err}")
            except Exception as err:
                print(f"Error for date {date}: {err}")

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

def save_to_excel(df: pd.DataFrame, filename: str):
    """
    Saves the DataFrame to an Excel file.

    Args:
        df (pd.DataFrame): DataFrame to save.
        filename (str): Filename for the Excel file.
    """
    df.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

In [6]:
sample_date = '20241204'

In [7]:
with requests.Session() as session:
    session.headers.update(HEADERS)
    payload = PAYLOAD_TEMPLATE.copy()
    payload["trdDd"] = sample_date

    response = session.post(API_URL, data=payload)

In [8]:
response.content

b'{"output":[{"ISU_CD":"KR4239VC0561","ISU_SRT_CD":"239VC056","ISU_NM":"HD\xed\x95\x9c\xea\xb5\xad\xec\xa1\xb0\xec\x84\xa0 C 202412    84,000(  10)","TDD_CLSPRC":"-","FLUC_TP_CD":"0","CMPPREVDD_PRC":"-","TDD_OPNPRC":"-","TDD_HGPRC":"-","TDD_LWPRC":"-","IMP_VOLT":"48.20","NXTDD_BAS_PRC":"111,000.00","ACC_TRDVOL":"0","ACC_TRDVAL":"0","ACC_OPNINT_QTY":"0","SECUGRP_ID":"OP"},{"ISU_CD":"KR4239VC0546","ISU_SRT_CD":"239VC054","ISU_NM":"HD\xed\x95\x9c\xea\xb5\xad\xec\xa1\xb0\xec\x84\xa0 C 202412    88,000(  10)","TDD_CLSPRC":"-","FLUC_TP_CD":"0","CMPPREVDD_PRC":"-","TDD_OPNPRC":"-","TDD_HGPRC":"-","TDD_LWPRC":"-","IMP_VOLT":"48.20","NXTDD_BAS_PRC":"107,000.00","ACC_TRDVOL":"0","ACC_TRDVAL":"0","ACC_OPNINT_QTY":"0","SECUGRP_ID":"OP"},{"ISU_CD":"KR4239VC0504","ISU_SRT_CD":"239VC050","ISU_NM":"HD\xed\x95\x9c\xea\xb5\xad\xec\xa1\xb0\xec\x84\xa0 C 202412    92,000(  10)","TDD_CLSPRC":"-","FLUC_TP_CD":"0","CMPPREVDD_PRC":"-","TDD_OPNPRC":"-","TDD_HGPRC":"-","TDD_LWPRC":"-","IMP_VOLT":"48.20","NXTDD_

In [9]:
dd = response.json()

거래소 주식 옵션 전체로 조회하는거 막혔음. 나만 막힌게 아니고 전부. 

하지만 개별 underlying 종목을 설정해서 조회할 경우 불러올 수 있음. 딱 1년치만 불러오자. 


### 개별 주식 옵션 데이터 불러오기 테스트

또 막힐 수 있으니까 일단 삼성만 1년치 불러오고 나머지는 다시 하는걸로? 

In [10]:
# 거래소 주식 옵션 종목들 조회 

with requests.Session() as session:
    session.headers.update(HEADERS_OPTION_HOME)
    payload = PAYLOAD_OPTION_HOME.copy()
    response = session.post(API_URL, data=payload)

In [11]:
available_underlyings = pd.DataFrame(response.json()['output'])

In [12]:
available_underlyings.head()

,value,name
0,KRDRVOPEQU,전체
1,KRDRVOPS39,HD한국조선 옵션
2,KRDRVOPS49,HD현대인프 옵션
3,KRDRVOPSGH,HD현대일렉 옵션
4,KRDRVOPSFA,HMM 옵션


In [13]:
len(available_underlyings)

59

In [14]:
avail_under_code_list = available_underlyings['value'].tolist()[1:]

In [15]:
samsung_elec = 'KRDRVOPS11'

In [30]:
# 개별 종목 call 되나 확인 

with requests.Session() as session:
    session.headers.update(HEADERS)
    payload = PAYLOAD_TEMPLATE2.copy()
    
    payload['trdDd'] = sample_date
    payload['trdDdBox1'] = sample_date
    payload['trdDdBox2'] = sample_date
    
    payload["ulyId"] = samsung_elec

    response = session.post(API_URL, data=payload)

In [31]:
dd1 = pd.DataFrame(response.json()['output'])
len(dd1)

168

In [32]:
sample_date

'20241204'

In [33]:
# 개별 종목 call, 기간 줘도 되나 확인. 

with requests.Session() as session:
    session.headers.update(HEADERS)
    payload = PAYLOAD_TEMPLATE2.copy()
    
    payload['trdDd'] = sample_date
    payload['trdDdBox1'] = '20241202'
    payload['trdDdBox2'] = sample_date
    
    payload["ulyId"] = samsung_elec

    response = session.post(API_URL, data=payload)

In [34]:
dd2 = pd.DataFrame(response.json()['output'])

In [35]:
len(dd2)

168

In [36]:
# trdDdBox1, trdDdBox2 는 그냥 deprecated된 legacy parameter로 보인다. 

In [ ]:
# 개별 종목 call, 기간 줘도 되나 확인. 

with requests.Session() as session:
    session.headers.update(HEADERS)
    payload = PAYLOAD_TEMPLATE2.copy()
    
    payload['trdDd'] = "" # 빠지면 안됨.
    payload['trdDdBox1'] = '20241202'
    payload['trdDdBox2'] = sample_date
    
    payload["ulyId"] = samsung_elec

    response = session.post(API_URL, data=payload)

In [38]:
dd3 = pd.DataFrame(response.json()['output'])

In [39]:
len(dd3)

0

### 개별 주식 옵션 데이터 불러와서 합치기

일단 테스트로 삼성만

In [46]:
def fetch_option_data1(session: requests.Session, uly_code: str, trade_date: str,) -> pd.DataFrame:
    payload = PAYLOAD_TEMPLATE.copy()
    
    payload["trdDd"] = trade_date
    payload['trdDdBox1'] = trade_date
    payload['trdDdBox2'] = trade_date

    payload["ulyId"] = uly_code

    response = session.post(API_URL, data=payload)
    response.raise_for_status()

    data = response.json()

    if "output" not in data:
        return pd.DataFrame()

    return pd.DataFrame(data["output"])

def scrape_krx_option_data1(start_date: str, end_date: str, ulycode_list: list) -> pd.DataFrame:
    dates = generate_date_range(start_date, end_date)
    
    all_data = []
    for uly_code in ulycode_list:
        with requests.Session() as session:
            session.headers.update(HEADERS)
            for date in tqdm(dates, desc="Fetching data"):
                try:
                    daily_data = fetch_option_data(session, uly_code, date)
                    if not daily_data.empty:
                        daily_data['Trade_Date'] = datetime.strptime(date, "%Y%m%d").date()
                        daily_data['Uly_Code'] = uly_code
                        all_data.append(daily_data)
                    time.sleep(3)  # Delay to mimic human behavior
                except requests.HTTPError as http_err:
                    print(f"HTTP error for date {date}: {http_err}")
                except Exception as err:
                    print(f"Error for date {date}: {err}")

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()
        


In [47]:
start = '2024-01-01'
end = '2024-12-04'

samsung_list = [samsung_elec]

In [48]:
samsung_df = scrape_krx_option_data1(start, end, samsung_list)

Fetching data: 100%|██████████| 243/243 [49:48<00:00, 12.30s/it] 


In [50]:
samsung_df.to_pickle(DATA_PATH / 'samsung_option.pkl')

In [49]:
len(samsung_df)

35978

In [131]:
samsung_df = pd.read_pickle(DATA_PATH / 'samsung_option.pkl')

In [132]:
samsung_df.columns

Index(['ISU_CD', 'ISU_SRT_CD', 'ISU_NM', 'TDD_CLSPRC', 'FLUC_TP_CD',
       'CMPPREVDD_PRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'IMP_VOLT',
       'NXTDD_BAS_PRC', 'ACC_TRDVOL', 'ACC_TRDVAL', 'ACC_OPNINT_QTY',
       'SECUGRP_ID', 'Trade_Date', 'Uly_Code'],
      dtype='object')

In [133]:
# strike price 등의 정보를 이름에서 파싱 

import re

def parse_option_name(option_name):
    pattern = r"(?P<underlying>[\w가-힣]+)\s+(?P<call_or_put>[CP])\s+(?P<expiration>\d{6})\s+(?P<strike>[\d,]+)\(\s*(?P<multiplier>\d+)\)"

    # Match the pattern
    match = re.match(pattern, option_name)

    # Extract data if a match is found
    if match:
        data = {
            "underlying": match.group("underlying"),
            "call_or_put": match.group("call_or_put"),
            "expiration": match.group("expiration"),
            "strike": float(match.group("strike").replace(",", "")),
            "multiplier": int(match.group("multiplier")),
        }

        return data
        
    else:
        data = {
            "underlying": None,
            "call_or_put": None,
            "expiration": None,
            "strike": None,
            "multiplier": None,
        }

        return data

In [134]:
parsed_data = samsung_df['ISU_NM'].apply(parse_option_name)

In [135]:
parsed_df = pd.DataFrame(parsed_data.tolist())

In [136]:
samsung_df = pd.concat([samsung_df, parsed_df], axis=1)

In [137]:
column_to_name = {
    'ISU_CD': 'option_sid(full)',  # 종목코드(full)
    'ISU_SRT_CD': 'option_sid(short)',  # 종목코드(short)
    'ISU_NM': 'option_name',  # 종목명
    'TDD_CLSPRC': 'close_price',  # 종가
    'FLUC_TP_CD': 'up_or_down',  # 등락구분
    'CMPPREVDD_PRC': 'price_change',  # 대비
    'TDD_OPNPRC': 'open_price',  # 시가
    'TDD_HGPRC': 'high_price',  # 고가
    'TDD_LWPRC': 'low_price',  # 저가
    'IMP_VOLT': 'im_vol',  # 내재변동성
    'NXTDD_BAS_PRC': 'next_day_base_price',  # 익일기준가
    'ACC_TRDVOL': 'trade_volume',  # 거래량
    'ACC_TRDVAL': 'trade_value',  # 거래대금
    'ACC_OPNINT_QTY': 'open_interest_quantity',  # 미결제약정수량
    'SECUGRP_ID': 'security_type',  # 증권유형
    'Trade_Date': 'trade_date',  # 거래일자
    'Uly_Code': 'underlying_code',  # 기초자산코드
}

In [138]:
samsung_df.rename(columns=column_to_name, inplace=True)

In [139]:
float_cols = [
    'close_price', 'price_change', 'open_price', 'high_price', 'low_price',
    'im_vol', 'next_day_base_price', 'trade_volume', 'trade_value',
    'open_interest_quantity', 
    'expiration', 
    'up_or_down',
    # 'strike', 
    # 'multiplier',
]

In [140]:
for float_col in float_cols:
    print(f'converting {float_col} to numeric')
    samsung_df[float_col] = samsung_df[float_col].str.replace('-', '')
    samsung_df[float_col] = samsung_df[float_col].str.replace(',', '')
    samsung_df[float_col] = pd.to_numeric(samsung_df[float_col], errors='raise')

converting close_price to numeric
converting price_change to numeric
converting open_price to numeric
converting high_price to numeric
converting low_price to numeric
converting im_vol to numeric
converting next_day_base_price to numeric
converting trade_volume to numeric
converting trade_value to numeric
converting open_interest_quantity to numeric
converting expiration to numeric
converting up_or_down to numeric


In [143]:
samsung_df['trade_date'] = pd.to_datetime(samsung_df['trade_date'])

In [144]:
samsung_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35978 entries, 0 to 35977
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   option_sid(full)        35978 non-null  object        
 1   option_sid(short)       35978 non-null  object        
 2   option_name             35978 non-null  object        
 3   close_price             4871 non-null   float64       
 4   up_or_down              35978 non-null  int64         
 5   price_change            4871 non-null   float64       
 6   open_price              4871 non-null   float64       
 7   high_price              4871 non-null   float64       
 8   low_price               4871 non-null   float64       
 9   im_vol                  35978 non-null  float64       
 10  next_day_base_price     35570 non-null  float64       
 11  trade_volume            35978 non-null  int64         
 12  trade_value             35978 non-null  int64 

데이터를 처리하기 쉬운 형태로 변환

In [145]:
samsung_df.columns

Index(['option_sid(full)', 'option_sid(short)', 'option_name', 'close_price',
       'up_or_down', 'price_change', 'open_price', 'high_price', 'low_price',
       'im_vol', 'next_day_base_price', 'trade_volume', 'trade_value',
       'open_interest_quantity', 'security_type', 'trade_date',
       'underlying_code', 'underlying', 'call_or_put', 'expiration', 'strike',
       'multiplier'],
      dtype='object')

In [146]:
index_cols = [
    'underlying',
    'call_or_put',
    'expiration',
    'trade_date',
    'strike',
]

In [147]:
value_cols = [
    'close_price',
    'open_price',
    'high_price',
    'low_price',
    'im_vol',
    'next_day_base_price',
    'trade_volume',
    'trade_value',
    'open_interest_quantity',
]

In [148]:
samsung_final_df = samsung_df[index_cols + value_cols].copy()
samsung_final_df.set_index(index_cols, inplace=True)


In [149]:
samsung_final_df

close_price  open_price  \
underlying call_or_put expiration trade_date strike                             
삼성전자       C           202412     2024-12-04 45000.0          NaN         NaN   
                                             46000.0          NaN         NaN   
                                             47000.0          NaN         NaN   
                                             48000.0          NaN         NaN   
                                             49000.0          NaN         NaN   
...                                                           ...         ...   
           P           202412     2024-01-02 80000.0          NaN         NaN   
                                             84000.0          NaN         NaN   
                                             88000.0          NaN         NaN   
                                             92000.0          NaN         NaN   
                                             96000.0          NaN         NaN   

                                                      high_price  low_price  \
underlying call_or_put expiration trade_date strike                           
삼성전자       C           202412     2024-12-04 45000.0         NaN        NaN   
                                             46000.0         NaN        NaN   
                                             47000.0         NaN        NaN   
                                             48000.0         NaN        NaN   
                                             49000.0         NaN        NaN   
...                                                          ...        ...   
           P           202412     2024-01-02 80000.0         NaN        NaN   
                                             84000.0         NaN        NaN   
                                             88000.0         NaN        NaN   
                                             92000.0         NaN        NaN   
                                             96000.0         NaN        NaN   

                                                      im_vol  \
underlying call_or_put expiration trade_date strike            
삼성전자       C           202412     2024-12-04 45000.0    33.7   
                                             46000.0    33.7   
                                             47000.0    33.7   
                                             48000.0    33.7   
                                             49000.0    33.7   
...                                                      ...   
           P           202412     2024-01-02 80000.0    19.1   
                                             84000.0    19.1   
                                             88000.0    19.1   
                                             92000.0    19.1   
                                             96000.0    19.1   

                                                      next_day_base_price  \
underlying call_or_put expiration trade_date strike                         
삼성전자       C           202412     2024-12-04 45000.0               8100.0   
                                             46000.0               7100.0   
                                             47000.0               6100.0   
                                             48000.0               5200.0   
                                             49000.0               4200.0   
...                                                                   ...   
           P           202412     2024-01-02 80000.0               5100.0   
                                             84000.0               7200.0   
                                             88000.0               9700.0   
                                             92000.0              12600.0   
                                             96000.0              15600.0   

                                                      trade_volume  \
underlying call_or_put expiration trade_date strike                

In [151]:
samsung_final_df.to_pickle(DATA_PATH / 'samsung_option_final.pkl')

In [150]:
START_DATE = '2021-01-01'
END_DATE = '2024-12-05'


### New Prompt

Implement retry strategy in this code. 
Retry strategy should:
- when there's bad requests like 400, step back and wait and retry. 

You should suggest me additional measure to avoid getting blocked. Is there a free VPN service that I can use to hide my identity? 

Also, to avoid losing all the data when I get blocked, write the code to:

- start from end date to start date
- for each date scraped, save it in the data by appending the data (not overwriting). You should use h5py to save it in h5 format to append the data. 
- periodically (per 100 scrape) copy that h5 file to somewhere else for me to safely use it without racing condition. ( I will copy the copy of the h5 and use it at somewhere else)

In [ ]:
# main.py

import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from typing import List
from tqdm import tqdm
import h5py
import os
import shutil
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

from zoneinfo import ZoneInfo  # Python 3.9 이상

# Import configurations
from krx_config import API_URL, HEADERS, PAYLOAD_TEMPLATE



# Constants for backup
BACKUP_INTERVAL = 100  # Backup after every 100 successful scrapes
H5_FILE = DATA_PATH / "krx_option_data.h5"

def generate_date_range(start_date: str, end_date: str) -> List[str]: 
    """
    종료일(end_date)부터 시작일(start_date)까지의 비주말(평일) 날짜를 YYYYMMDD 형식으로 반환합니다.
    날짜는 한국 서울 시간대 기준입니다.

    Args:
        start_date (str): 시작 날짜 ('YYYY-MM-DD' 형식).
        end_date (str): 종료 날짜 ('YYYY-MM-DD' 형식).

    Returns:
        List[str]: 내림차순으로 정렬된 비주말 날짜 리스트 ('YYYYMMDD' 형식).
    """
    tz = ZoneInfo('Asia/Seoul')
    start = datetime.strptime(start_date, "%Y-%m-%d").replace(tzinfo=tz)
    end = datetime.strptime(end_date, "%Y-%m-%d").replace(tzinfo=tz)
    delta = end - start
    date_list = []
    for i in range(delta.days + 1):
        current_date = end - timedelta(days=i)
        if current_date.weekday() < 5:  # 0-4는 월요일~금요일
            date_list.append(current_date.strftime("%Y%m%d"))
    return date_list

def setup_session() -> requests.Session:
    """
    Sets up a requests session with headers and retry strategy.
    """
    session = requests.Session()
    session.headers.update(HEADERS)
    
    retry_strategy = Retry(
        total=5,
        status_forcelist=[400, 429, 500, 502, 503, 504],
        method_whitelist=["POST"],
        backoff_factor=1
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    return session

def fetch_option_data(session: requests.Session, trade_date: str) -> pd.DataFrame:
    """
    Fetches option data for a specific trade date.
    """
    payload = PAYLOAD_TEMPLATE.copy()
    payload["trdDd"] = trade_date

    response = session.post(API_URL, data=payload)
    response.raise_for_status()

    data = response.json()

    if "output" not in data:
        return pd.DataFrame()

    return pd.DataFrame(data["output"])

def save_data_h5(df: pd.DataFrame, filename: str):
    """
    Appends DataFrame to an HDF5 file.
    """
    with pd.HDFStore(filename, mode='a') as store:
        store.append('option_data', df, format='table', data_columns=True)

def backup_h5_file(source: str, backup_path: Path):
    """
    Copies the HDF5 file to the backup directory with a timestamp.
    """
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    backup_path = BACKUP_PATH / f"krx_option_data_backup_{timestamp}.h5"
    shutil.copy(source, backup_path)
    print(f"Backup created at {backup_path}")

def scrape_krx_option_data(start_date: str, end_date: str) -> None:
    """
    Scrapes KRX option data between start_date and end_date and saves to HDF5.
    Implements retry and backup strategies.
    """
    dates = generate_date_range(start_date, end_date)
    success_count = 0

    session = setup_session()
    
    for date in tqdm(dates, desc="Fetching data"):
        try:
            daily_data = fetch_option_data(session, date)
            if not daily_data.empty:
                daily_data['Trade_Date'] = datetime.strptime(date, "%Y%m%d").date()
                save_data_h5(daily_data, H5_FILE)
                success_count += 1
                
                if success_count % BACKUP_INTERVAL == 0:
                    backup_h5_file(H5_FILE, BACKUP_PATH)
            
            time.sleep(1)  # Delay to mimic human behavior
        except requests.HTTPError as http_err:
            print(f"HTTP error for date {date}: {http_err}")
            time.sleep(5)  # Wait before retrying
        except Exception as err:
            print(f"Error for date {date}: {err}")
            time.sleep(5)  # Wait before continuing
    
    # Final backup after completion
    backup_h5_file(H5_FILE, BACKUP_PATH)


In [ ]:

def main():
    # Example usage
    START_DATE = "2024-12-01"
    END_DATE = "2024-12-06"
    
    print(f"Scraping KRX option data from {START_DATE} to {END_DATE}...")
    scrape_krx_option_data(START_DATE, END_DATE)
    print("Scraping completed.")

if __name__ == "__main__":
    main()
